## Imports ##

In [38]:
import pandas as pd
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import torch.nn as nn
from src.utils import OurModel

import wandb

In [39]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

## Batch preperation ##

In [41]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features (19)
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split (X, y, test_size=0.1, random_state=32) # 90% train, 10% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=32) # test -> 50% validation, 50% final test

In [42]:
# Creating Pytorch Tensors
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [43]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [44]:
batch_size = 1024 # bigger batch size because of 193000 samples (x^2)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

## Model Training ##

In [45]:
ml = OurModel()

In [46]:
# model configuration 1
config = nn.Sequential(
    nn.Linear(COVID_data.shape[1]-1, 8),
    nn.ReLU(),
    nn.Linear(8, 32),
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
)
ml.add_configuration(config)

In [47]:
# model configuration 2
config = nn.Sequential(
            nn.Linear(19, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),      #
            nn.ReLU(),
            nn.Linear(32, 1)
        )
ml.add_configuration(config)

In [48]:
#GPU optimization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
ml.to(device)

cuda


OurModel(
  (net): Sequential(
    (0): Linear(in_features=19, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [49]:
criterion = nn.BCEWithLogitsLoss() # combines a Sigmoid layer and the BCELoss in one single class.
optimizer = optim.Adam(ml.parameters(), lr=0.001)

In [50]:
# # Model Training
epochs = 20  # number of training iterations over the full dataset

for epoch in range(epochs):
    ml.train()  # set model to training mode (enables dropout, batchnorm updates)
    total_loss = 0  # accumulates total loss across all batches
    correct = 0     # counts correct predictions
    total = 0       # counts total samples

    # iterate over mini-batches from the training DataLoader
    for inputs, true in train_loader:
        # move input tensors and labels to the target device (GPU/CPU)
        inputs, true = inputs.to(device), true.to(device)
        # ensure labels have shape [batch_size, 1] to match model output
        true = true.unsqueeze(1)
        # forward pass: compute model predictions (logits)
        y_pred = ml(inputs)
        # compute loss between predictions and true labels
        loss = criterion(y_pred, true)
        # clear old gradients before backpropagation
        optimizer.zero_grad()
        # backward pass: compute gradients
        loss.backward()
        # update model parameters based on gradients
        optimizer.step()
        # accumulate batch loss for average calculation later
        total_loss += loss.item()
        # apply sigmoid to logits to get probabilities (0–1 range)
        preds = torch.sigmoid(y_pred)
        # convert probabilities to binary predictions (0 or 1)
        predicted = (preds > 0.5).float()
        # count how many predictions match the true labels
        correct += (predicted == true).sum().item()
        # count total samples processed
        total += true.size(0)

    # compute average loss per batch
    avg_loss = total_loss / len(train_loader)

    # compute overall accuracy for this epoch
    accuracy = correct / total

    print(f"Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.4f} | Accuracy: {accuracy:.4f}")



Epoch [1/20] | Loss: 0.6834 | Accuracy: 0.5733
Epoch [2/20] | Loss: 0.6614 | Accuracy: 0.6029


KeyboardInterrupt: 

## Sweep ##

In [51]:
wandb.login() #login to wandb account

True

In [52]:
#Hyperparameter configuration
#Random search
sweep_config = {
    "method": "random",
    "metric": {"name": "loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"min": 0.0001, "max": 0.01},
        # "epochs": {"min": 2, "max": 20},
        "hidden_size_1": {"values": [248, 512]},
        "hidden_size_2": {"values": [64, 128, 248]},
        "hidden_size_3": {"values": [64, 128, 248]},
        "hidden_size_4": {"values": [64, 128, 248]},
        "hidden_size_5": {"values": [64, 128, 248]},
        "hidden_size_6": {"values": [64, 128, 248]},
        "n_layers": {"values": [2, 6]},
        # "dropout": {"min": 0.0, "max": 0.5},
        "dropout": {"values": [0]},
        "optimizer": {"values": ["Adam"]},
        "activation": {"values": ["relu", "tanh"]},
        "batch_size": {"values": [2048]},
    }
}

In [53]:
#Grid search
# sweep_config = {
#     "method": "grid",
#     "metric": {"name": "loss", "goal": "minimize"},
#     "parameters": {
#         "learning_rate": {"values": [0.001, 0.005]},  # changed from range to discrete values
#         "hidden_size_1": {"values": [64, 128]},
#         "hidden_size_2": {"values": [64, 128]},
#         "hidden_size_3": {"values": [64, 128]},
#         "hidden_size_4": {"values": [64, 128]},
#         # "hidden_size_5": {"values": [64, 128]},
#         # "hidden_size_6": {"values": [64, 128]},
#         "n_layers": {"values": [2, 4]},
#         "dropout": {"values": [0.0, 0.2, 0.4, 0.5]},  # converted range to discrete values
#         "optimizer": {"values": ["Adam"]},
#         "activation": {"values": ["relu", "tanh"]},
#         "batch_size": {"values": [4096]},
#     }
# }

In [54]:
def train():
    wandb.init()  # initialize Weights & Biases tracking
    CONFIG = wandb.config

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=CONFIG.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=CONFIG.batch_size, shuffle=False)

    # Hidden layer configuration from W&B sweep parameters
    hl_sizes = [
        CONFIG.hidden_size_1, CONFIG.hidden_size_2, CONFIG.hidden_size_3,
        CONFIG.hidden_size_4, CONFIG.hidden_size_5, CONFIG.hidden_size_6,
    ]

    # Build model dynamically based on sweep config
    sw_nn = OurModel()
    sw_nn.sweeping_build(
        COVID_data.shape[1] - 1, 1,
        n_layers=CONFIG.n_layers,
        hidden_size=hl_sizes,
        activation_f=CONFIG.activation,
        dropout=CONFIG.dropout
    )

    # Optimizer & loss
    optimizer = getattr(torch.optim, CONFIG.optimizer)(sw_nn.parameters(), lr=CONFIG.learning_rate)
    criterion = nn.BCEWithLogitsLoss()

    # Device setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    sw_nn.to(device)

    # Training loop
    for epoch in range(10):
        sw_nn.train()
        total_loss, correct, total = 0, 0, 0

        for inputs, true in train_loader:
            inputs, true = inputs.to(device), true.to(device)
            true = true.unsqueeze(1)

            optimizer.zero_grad()
            y_pred = sw_nn(inputs)
            loss = criterion(y_pred, true)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = torch.sigmoid(y_pred)
            predicted = (preds > 0.5).float()
            correct += (predicted == true).sum().item()
            total += true.size(0)

        avg_train_loss = total_loss / len(train_loader)
        train_acc = correct / total

        # Validation
        sw_nn.eval()
        val_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, true in val_loader:
                inputs, true = inputs.to(device), true.to(device)
                true = true.unsqueeze(1)
                y_preds = sw_nn(inputs)
                loss = criterion(y_preds, true)
                val_loss += loss.item()
                preds = torch.sigmoid(y_preds)
                predicted = (preds > 0.5).float()
                correct += (predicted == true).sum().item()
                total += true.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_acc = correct / total

        # Log metrics to Weights & Biases
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "val_accuracy": val_acc
        })


In [55]:
sweep_id = wandb.sweep(sweep=sweep_config, project="xbukovinam-faculty-of-informatics-and-infromation-techno")
wandb.agent(sweep_id, function=train)

Create sweep with ID: wq1yvade
Sweep URL: https://wandb.ai/xbukovinam-faculty-of-informatics-and-infromation-techno/xbukovinam-faculty-of-informatics-and-infromation-techno/sweeps/wq1yvade


wandb: Agent Starting Run: tp2cn3hm with config:
wandb: 	activation: relu
wandb: 	batch_size: 2048
wandb: 	dropout: 0
wandb: 	hidden_size_1: 248
wandb: 	hidden_size_2: 64
wandb: 	hidden_size_3: 64
wandb: 	hidden_size_4: 64
wandb: 	hidden_size_5: 64
wandb: 	hidden_size_6: 248
wandb: 	learning_rate: 0.0043654429010920376
wandb: 	n_layers: 2
wandb: 	optimizer: Adam


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▂▁▁▁▁▁▁▁▁
val_accuracy,▁▇█▇▇▇▇██▇
val_loss,█▂▁▁▁▁▂▁▁▁
epoch,10
train_loss,0.63787
val_accuracy,0.63613
val_loss,0.63499


wandb: Agent Starting Run: hmba5cu5 with config:
wandb: 	activation: relu
wandb: 	batch_size: 2048
wandb: 	dropout: 0
wandb: 	hidden_size_1: 512
wandb: 	hidden_size_2: 128
wandb: 	hidden_size_3: 128
wandb: 	hidden_size_4: 248
wandb: 	hidden_size_5: 128
wandb: 	hidden_size_6: 64
wandb: 	learning_rate: 0.00398667666011733
wandb: 	n_layers: 2
wandb: 	optimizer: Adam


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▇█████▇█
val_loss,█▃▃▁▁▁▁▁▁▁
epoch,10
train_loss,0.63841
val_accuracy,0.6391
val_loss,0.63349


wandb: Agent Starting Run: 2nx37y4j with config:
wandb: 	activation: tanh
wandb: 	batch_size: 2048
wandb: 	dropout: 0
wandb: 	hidden_size_1: 512
wandb: 	hidden_size_2: 64
wandb: 	hidden_size_3: 128
wandb: 	hidden_size_4: 128
wandb: 	hidden_size_5: 128
wandb: 	hidden_size_6: 128
wandb: 	learning_rate: 0.009939359419175692
wandb: 	n_layers: 6
wandb: 	optimizer: Adam


epoch,▁▂▃▃▄▅▆▆▇█
train_loss,█▂▂▁▁▁▁▁▁▁
val_accuracy,▃▆▄▆▇▁▁▆▇█
val_loss,█▆▅▂▂▇▄▃▂▁
epoch,10
train_loss,0.63755
val_accuracy,0.64095
val_loss,0.63248


wandb: Ctrl + C detected. Stopping sweep.
